In [ ]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import requests
from openai import OpenAI
import tiktoken

In [3]:
ai_client = OpenAI()
es_client = Elasticsearch("http://127.0.0.1:9200")
es_client.info()

ObjectApiResponse({'name': 'b600c7454380', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'ecuqdC8_T1-xBEICGnC2FA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
def get_faq_data():
    docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()
    
    documents = []
    
    for course in documents_raw:
        course_name = course['course']
    
        for doc in course['documents']:
            doc['course'] = course_name
            documents.append(doc)
    return documents

In [5]:
def elastic_index(documents):
    index_settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "text"},
                "question": {"type": "text"},
                "course": {"type": "keyword"} 
            }
        }
    }
    index_name = "course_questions"
    es_client.indices.create(index=index_name, body=index_settings)
    
    for doc in tqdm(documents):
        es_client.index(index=index_name, document=doc)

In [6]:
def elastic_search(query,size=5):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    index_name = "course_questions"
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response["hits"]["hits"]:
        result_docs.append(hit["_source"])
    return result_docs

In [7]:
query = "How do copy a file to a Docker container?"

elastic_search(query,3)

In [11]:
def build_prompt(query, search_results):
    context = ""
    context_template = """
Q: {question}
A: {text}
""".strip()

    for doc in search_results:
        context += context_template.format(question=doc["question"],text=doc["text"])
        context += "\n\n"

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [12]:
len(build_prompt(query,elastic_search(query,3)))

1446

In [27]:
def llm(prompt, model):
    try:
        chat_completion = ai_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )
        return chat_completion.choices[0].message.content
    except:
        return None

In [28]:
search_results = elastic_search(query,3)
prompt = build_prompt(query, search_results)
llm(prompt, "gpt-4o")

In [30]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [31]:
encoding.decode_single_token_bytes(63842)

b"You're"